<a href="https://www.kaggle.com/code/luansilva22/exploring-online-foods?scriptVersionId=185115998" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/luansilva22/exploring-online-foods?scriptVersionId=185115998" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div style="width:100%; height:50px; background-color: #FA8072; border-radius:5px 5px; display:flex; ">
    <h1 style="width:100%; font-weight: bold; font-family:Times New Roman; color:#FFF5EE; font-size:140%; font-family:Monospace; text-align:center; margin: auto">Online Food Ordering</h1>
</div>

<div>
    <h3 style="font-weight: bold; font-family:Times New Roman">Description:</h3>
    <p  style="font-size: 16px;font-family:Times New Roman">his project is based on the analysis of a dataset of online food orders, with the aim of exploring the relationships between demographic factors, location and customer behavior.</p>
</div>
<nav>
    <h3 style="font-weight: bold; font-family:Times New Roman">Attributes:</h3>
    <ul style="font-size: 16px;font-family:Times New Roman">
        <li><strong>Age:</strong> Age of the customer.</li>
        <li><strong>Gender:</strong> Gender of the customer.</li>
        <li><strong>Marital Status:</strong> Marital status of the customer.</li>
        <li><strong>Occupation:</strong> Occupation of the customer.</li>
        <li><strong>Monthly Income:</strong> Monthly income of the customer.</li>
        <li><strong>Educational Qualifications:</strong> Educational qualifications of the customer.</li>
        <li><strong>Family Size:</strong> Number of individuals in the customer's family.</li>
        <li><strong>Latitude:</strong> Latitude of the customer's location.</li>
        <li><strong>Longitude:</strong> Longitude of the customer's location.</li>
        <li><strong>Output:</strong> Current status of the order (e.g., pending, confirmed, delivered).</li>
        <li><strong>Feedback:</strong> Feedback provided by the customer after receiving the order.</li>
    </ul>
</nav>
<div style="font-family:Times New Roman">
    <h3 style="font-weight: bold; font-family:Times New Roman">Purpose:</h3>
    <p  style="font-size: 16px;font-family:Times New Roman">Explore whether there are purchasing relationships for the food sold, understanding the influence of location, age, gender, family size and occupation of the customer on the purchase of the product</p>
</div>

<div style="width:100%; height:50px; background-color: #FA8072; border-radius:5px 5px; display:flex; ">
    <h1 style="width:100%; font-weight: bold; font-family:Times New Roman; color:#FFF5EE; font-size:140%; font-family:Monospace; text-align:center; margin: auto">STEP 1 | Importing Data & Libraries</h1>
</div>

In [ ]:
!pip install pandas
!pip install matplotlib
!pip install geopy
!pip install seaborn

In [6]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
import warnings  # Suppress warnings

# HTML & Styles
from colorama import Fore, Style
from IPython.display import display, HTML


<div style="width:100%; height:50px; background-color: #FA8072; border-radius:5px 5px; display:flex; ">
    <h1 style="width:100%; font-weight: bold; font-family:Times New Roman; color:#FFF5EE; font-size:140%; font-family:Monospace; text-align:center; margin: auto">STEP 2 | Loading Data</h1>
</div>

In [9]:
df_Foods = pd.read_csv('https://www.kaggle.com/datasets/sudarshan24byte/online-food-dataset.csv')

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

<div style="width:100%; height:50px; background-color: #FA8072; border-radius:5px 5px; display:flex; ">
    <h1 style="width:100%; font-weight: bold; font-family:Times New Roman; color:#FFF5EE; font-size:140%; font-family:Monospace; text-align:center; margin: auto">Step 3 | Overview</h1>
</div>

In [ ]:
# df_Foods.drop(['Unnamed: 12'],inplace=True, axis=1)

In [ ]:
def PrintColor(text: str, color=Fore.CYAN, style=Style.BRIGHT):
    "Prints color outputs using colorama using a text F-string"
    print(style + color + text + Style.RESET_ALL)

def print_blue_large(text):
    PrintColor(text, Fore.BLUE + Style.BRIGHT)

def Heading(textLabels):
    length = len(textLabels) + 8
    print(f"\n{Style.BRIGHT}{Fore.RED}{'='*length}{Style.RESET_ALL}")
    print(f"{Style.BRIGHT}{Fore.RED} || {textLabels}  {Style.RESET_ALL}")
    print(f"{Style.BRIGHT}{Fore.RED}{'='*length}{Style.RESET_ALL}")

# Function | Data Overview
def fDataOverview(data):

    # First 5 items of the DataFrame
    Heading("First 5 items of the DataFrame:")
    display(HTML(data.head(5).to_html(index=False).replace('<table border="1" class="data">', '<table style="border: 2px solid green;">')))

    # Remove column
    # df_Foods.drop(['Unnamed: 12'], index=1,inplace=True)
    Heading("COLUMN REMOVED: Unnamed: 12")

    # Information columns
    Heading("Information columns")
    PrintColor(f"Info Dataset | THERE ARE NO NULL VALUES", Fore.CYAN)
    df_Foods.info()

    # Describe
    Heading("Describe Columns")
    display(HTML(df_Foods.describe().to_html(index=False).replace('<table border="1" class="data">', '<table style="border: 2px solid green;">')))

    # Duplicates rows found
    Heading("Duplicates rows found:")
    print(df_Foods.duplicated().sum())

    # Number of rows and columns
    Heading("Number of rows and columns:")
    print(df_Foods.shape)



In [ ]:
fDataOverview(df_Foods)

In [ ]:
def unique_values(train_dataset):
    df_UniqueValues = pd.DataFrame({
        'Name': train_dataset.columns,
        'Type': [train_dataset[col].dtype for col in train_dataset.columns],
        'Unique Values': [', '.join(map(str, train_dataset[col].unique()[0:7])) for col in train_dataset.columns]
    })
    display(df_UniqueValues)

Heading("Unique Values in Dataset:")
unique_values(df_Foods)

<h3 style="width:100%; font-weight: bold; font-family:Times New Roman; color:#0000ff; font-size:140%; font-family:Monospace; margin: auto">Reports</h3>

In [ ]:
print_blue_large(f"\n1)The Unnamed: 12 column has been removed from the dataframe. \n2)There are no null values. \n3)Duplicate values: {df_Foods.duplicated().sum()}. \n4)Total number of rows and columns {df_Foods.shape}")

<div style="width:100%; height:50px; background-color: #FA8072; border-radius:5px 5px; display:flex; ">
    <h1 style="width:100%; font-weight: bold; font-family:Times New Roman; color:#FFF5EE; font-size:140%; font-family:Monospace; text-align:center; margin: auto">Step 4 | Feature Engineering</h1>
</div>

In [ ]:
# Function to get a adresss based on latitude and longitude
def get_county_name(latitude, longitude):
    # Inicialized geocoder
    geolocator = Nominatim(user_agent="Online Food Order")
    # Concatenates latitude and longitude into a string in the format "latitude, longitude"
    location = f"{latitude}, {longitude}"
    # Use the geocoder to obtain the address
    address = geolocator.reverse(location)
    # Checks if the address was found
    if address:
        # Extracts the city name from the 'address' subdictionary
        county = address.raw['address'].get('county', '')  # If 'city' does not exist, returns an empty string
        return county
    else:
        return None

# Apply the function to the DataFrame to obtain city names
df_Foods['county'] = df_Foods.apply(lambda row: get_county_name(row['latitude'], row['longitude']), axis=1)

In [ ]:
Heading("County column created based on latitude and longitude: count")

<h3 style="width:100%; font-weight: bold; font-family:Times New Roman; color:#0000ff; font-size:140%; font-family:Monospace; margin: auto">Reports</h3>

In [ ]:
print_blue_large(f"\nCreate Column: {df_Foods['county'].name}")

<div style="width:100%; height:50px; background-color: #FA8072; border-radius:5px 5px; display:flex; ">
    <h1 style="width:100%; font-weight: bold; font-family:Times New Roman; color:#FFF5EE; font-size:140%; font-family:Monospace; text-align:center; margin: auto">Step 5 | EDA Analysis</h1>
</div>

<h3 style="width:100%; font-weight: bold; font-family:Times New Roman; color:#808080; font-size:140%; font-family:Monospace; margin: auto">Summary</h3>

In [ ]:
def frequencyNumber(ColumnName, Data):

    counts = Data[ColumnName].value_counts()

    # Sort age counts in descending order
    counts = counts.sort_values(ascending=False)

    # Normalize age counts for color gradient
    max_value = counts.max()
    normalized_counts = counts / max_value

    # Create color gradient from dark to light Brown
    cmap = plt.cm.get_cmap('Oranges')
    colors = cmap(normalized_counts)

    # Create a horizontal bar chart
    fig, ax = plt.subplots()
    bars = ax.barh(counts.index, counts.values, color=colors)

    for rect, value in zip(bars, counts.values):
        _x = rect.get_width()  # Get bar width
        _y = rect.get_y() + rect.get_height() / 2
        ax.text(_x + 1, _y, f" {value}", va='center', ha='left', fontsize=8)
        
    # Remove border
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

    average_age = Data[ColumnName].mean()
    ax.axhline(y=average_age, color='blue', linestyle='--', label=f'Average {ColumnName}')
    average_max = Data[ColumnName].max()
    ax.axhline(y=average_max, color='gray', linestyle='-', label=f'Max {ColumnName}')
    average_min = Data[ColumnName].min()
    ax.axhline(y=average_min, color='gray', linestyle='-', label=f'Min {ColumnName}')

    ax.text(78.0, average_age, f"Avg: {average_age:.2f}", va='center', ha='left', color='blue', fontsize=10)
    ax.text(78.0, average_max, f"Max: {average_max:.2f}", va='center', ha='left', color='gray', fontsize=10)
    ax.text(78.0, average_min, f"Min: {average_min:.2f}", va='center', ha='left', color='gray', fontsize=10)

    # Set labels and title
    plt.xlabel('Frequency of Orders')
    plt.ylabel(ColumnName)
    plt.title(f'Frequency of Orders by {ColumnName}')

    # Improve readability
    plt.yticks(rotation=0)

    plt.show()

In [ ]:
frequencyNumber('Age', df_Foods)

In [ ]:
frequencyNumber('Family size', df_Foods)

In [ ]:
def frequency(ColumnName, Dataset):

    counts = Dataset[ColumnName].value_counts()

    counts = counts.sort_values(ascending=True)

    max_value = counts.max()
    normalized_counts = counts / max_value

    cmap = plt.cm.get_cmap('Oranges')
    colors = cmap(normalized_counts)

    fig, ax = plt.subplots()
    bars = ax.barh(counts.index, counts.values, color=colors)

    for rect, value in zip(bars, counts.values):
        _x = rect.get_width()  # Get bar width
        _y = rect.get_y() + rect.get_height() / 2
        ax.text(_x + 1, _y, f" {value}", va='center', ha='left', fontsize=12)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

    plt.xlabel('Frequency of Orders')
    plt.ylabel(ColumnName)
    plt.title(f'Frequency of Orders by {ColumnName}')

    # Improve readability
    plt.yticks(rotation=0)

    plt.show()

In [ ]:
frequency('Gender', df_Foods)

In [ ]:
frequency('Marital Status', df_Foods)

In [ ]:
frequency('Occupation', df_Foods)

In [ ]:
frequency('Monthly Income', df_Foods)

In [ ]:
frequency('Educational Qualifications', df_Foods)

In [ ]:
frequency('county', df_Foods)

In [ ]:
frequency('Feedback', df_Foods)

In [ ]:
df_Foods.head(1)

<h3 style="width:100%; font-weight: bold; font-family:Times New Roman; color:#0000ff; font-size:140%; font-family:Monospace; margin: auto">Reports</h3>

In [ ]:
print_blue_large(f"\nFREQUENCY OF ORDERS BY AGE \nThe age group with the highest number of requests is {df_Foods['Age'].value_counts().idxmax() } years old, with an average of {df_Foods['Age'].mean()} years old.\nMinimun age:{df_Foods['Age'].min()} \nMaximum age:{df_Foods['Age'].max()}")
print_blue_large(f"\n\nFREQUENCY OF ORDERS BY FAMILY SIZE \nThe Family size group with the highest number of requests is {df_Foods['Family size'].value_counts().idxmax() } people, with an average of ~ {df_Foods['Family size'].mean()}.\nMinimun:{df_Foods['Family size'].min()} \nMaximum:{df_Foods['Family size'].max()}")

In [ ]:
print_blue_large(f"\nFREQUENCY OF ORDERS BY Gender \nThe Gender group with the highest number of requests is:\n{df_Foods[df_Foods['Gender'] == 'Male']['Gender'].value_counts()} \n{df_Foods[df_Foods['Gender'] == 'Female']['Gender'].value_counts()}")

In [ ]:
# IN PROGRESS

<h3 style="width:100%; font-weight: bold; font-family:Times New Roman; color:#808080; font-size:140%; font-family:Monospace; margin: auto">Filter by Gender</h3>

<div style="width:100%; height:50px; background-color: #FA8072; border-radius:5px 5px; display:flex; ">
    <h1 style="width:100%; font-weight: bold; font-family:Times New Roman; color:#FFF5EE; font-size:140%; font-family:Monospace; text-align:center; margin: auto">Step 6 | Outlier Dedection</h1>
</div>

# Step 7 | Numerical Feature Distribution

# Step 7.1 | Normalization

# Step 8 | Correlation Analysis

# Step 9 | Encoding

# Step 10 | Model Building

# Base Classifier

# Fitting Classifier

# Evaluating Classifier

# Storing Reults in DF and Visualizing the Results